In [1]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset


In [3]:

from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/heart_disease_cleaned.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
X = df.drop('target', axis=1)
Y = df['target'].values


In [11]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [15]:

# Reshape to (samples, height, width, channels) to simulate image input
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [17]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

In [18]:


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [19]:
# STEP 4: Channel shuffle
def channel_shuffle(x, groups):
    batchsize, num_channels, length = x.size()
    channels_per_group = num_channels // groups
    x = x.view(batchsize, groups, channels_per_group, length)
    x = x.transpose(1, 2).contiguous()
    x = x.view(batchsize, -1, length)
    return x

In [20]:

# STEP 5: ShuffleBlock and Model
class ShuffleBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, groups=2):
        super(ShuffleBlock1D, self).__init__()
        self.groups = groups
        self.group_conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1, groups=groups)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        self.dwconv = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1, groups=out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.group_conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=1, groups=groups)
        self.bn3 = nn.BatchNorm1d(out_channels)

    def forward(self, x):
        x = self.group_conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = channel_shuffle(x, self.groups)
        x = self.dwconv(x)
        x = self.bn2(x)
        x = self.group_conv2(x)
        x = self.bn3(x)
        return self.relu(x)

In [21]:
class ShuffleNet1D(nn.Module):
    def __init__(self, input_channels=1, input_length=13, num_classes=2):
        super(ShuffleNet1D, self).__init__()
        self.initial = nn.Sequential(
            nn.Conv1d(input_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU()
        )
        self.stage1 = ShuffleBlock1D(32, 64)
        self.stage2 = ShuffleBlock1D(64, 128)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.initial(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


In [22]:
# STEP 6: Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ShuffleNet1D().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [23]:
for epoch in range(20):
    model.train()
    total_loss = 0
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/20, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1/20, Loss: 0.5706
Epoch 2/20, Loss: 0.4772
Epoch 3/20, Loss: 0.4386
Epoch 4/20, Loss: 0.4012
Epoch 5/20, Loss: 0.3908
Epoch 6/20, Loss: 0.3737
Epoch 7/20, Loss: 0.3620
Epoch 8/20, Loss: 0.3470
Epoch 9/20, Loss: 0.3291
Epoch 10/20, Loss: 0.3176
Epoch 11/20, Loss: 0.3166
Epoch 12/20, Loss: 0.3132
Epoch 13/20, Loss: 0.2916
Epoch 14/20, Loss: 0.2613
Epoch 15/20, Loss: 0.2719
Epoch 16/20, Loss: 0.2659
Epoch 17/20, Loss: 0.2566
Epoch 18/20, Loss: 0.2542
Epoch 19/20, Loss: 0.2371
Epoch 20/20, Loss: 0.2697


In [24]:
from sklearn.metrics import accuracy_score, classification_report

model.eval()
preds, labels_all = [], []
with torch.no_grad():
    for batch in test_loader:
        x_batch, y_batch = batch
        x_batch = x_batch.to(device)
        outputs = model(x_batch)
        _, predicted = torch.max(outputs, 1)
        preds.extend(predicted.cpu().numpy())
        labels_all.extend(y_batch.numpy())

print("Accuracy:", accuracy_score(labels_all, preds))
print(classification_report(labels_all, preds))

Accuracy: 0.7880434782608695
              precision    recall  f1-score   support

           0       0.70      0.83      0.76        75
           1       0.86      0.76      0.81       109

    accuracy                           0.79       184
   macro avg       0.78      0.79      0.79       184
weighted avg       0.80      0.79      0.79       184



In [26]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


In [32]:
!pip install thop

In [37]:
import torch
import os
import time
import tempfile
import numpy as np
from thop import profile as thop_profile
from torch.profiler import profile, ProfilerActivity

def evaluate_pytorch_model_metrics(model, x_test, y_test, model_name="model_metrics", device=None):
    model.eval()
    results = {}

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    x_test = x_test.to(device)
    y_test = y_test.to(device)

    # Accuracy
    with torch.no_grad():
        outputs = model(x_test)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y_test).float().mean().item()

    results['accuracy'] = accuracy

    # Inference time
    start_time = time.time()
    with torch.no_grad():
        model(x_test)
    end_time = time.time()
    results['inference_time_seconds'] = end_time - start_time

    # Model size in bytes and MB
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pt") as tmp:
        torch.save(model.state_dict(), tmp.name)
        size_bytes = os.path.getsize(tmp.name)
        results['model_size_bytes'] = size_bytes
        results['model_size_MB'] = size_bytes / (1024 * 1024)
        os.unlink(tmp.name)  # delete temp file

    # Parameter count
    total_params = sum(p.numel() for p in model.parameters())
    results['total_parameters'] = total_params

    # FLOPs (optional basic estimate)
    try:
        from thop import profile as thop_profile
        dummy_input = torch.randn(1, *x_test.shape[1:]).to(device)
        flops, _ = thop_profile(model, inputs=(dummy_input,))
        results['FLOPs'] = flops
        results['GFLOPs'] = flops / 1e9
    except:
        results['FLOPs'] = "Requires thop package"
        results['GFLOPs'] = "Requires thop package"

    return results


In [38]:
metrics = evaluate_pytorch_model_metrics(model, X_test_tensor, y_test_tensor, model_name="shufflenet1d")
print(metrics)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool1d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
{'accuracy': 0.7880434989929199, 'inference_time_seconds': 0.020157337188720703, 'model_size_bytes': 93337, 'model_size_MB': 0.08901309967041016, 'total_parameters': 18114, 'FLOPs': 186304.0, 'GFLOPs': 0.000186304}
